In [50]:

# Adding some libraries to work
import numpy as np
from pylab import *
import pandas as pd
from numpy.matlib import repmat
import matplotlib 
import matplotlib.pyplot as plt
from scipy.io import loadmat
import time
import sys

%matplotlib notebook

sys.path.append('/home/codio/workspace/.guides/hf')
#from helper import *

print('You\'re running python %s' % sys.version.split(' ')[0])
print(pd.__version__)

You're running python 3.6.6
0.23.4


Download data from kaggle and store locally.
Reading data and storing into train and test

In [51]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
combine = [train_df, test_df]

In [52]:
print(train_df.columns.values)

['PassengerId' 'Survived' 'Pclass' 'Name' 'Sex' 'Age' 'SibSp' 'Parch'
 'Ticket' 'Fare' 'Cabin' 'Embarked']


In [53]:
train_df.info()
print('_'*40)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
________________________________________
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null

In [54]:
print("Before", train_df.shape, test_df.shape, combine[0].shape, combine[1].shape)

train_df = train_df.drop(['Ticket', 'Cabin'], axis=1)
test_df = test_df.drop(['Ticket', 'Cabin'], axis=1)
combine = [train_df, test_df]

"After", train_df.shape, test_df.shape, combine[0].shape, combine[1].shape

Before (891, 12) (418, 11) (891, 12) (418, 11)


('After', (891, 10), (418, 9), (891, 10), (418, 9))

Cleaning data, as it has null and infinity entries

In [55]:
train_df["Age"] = train_df["Age"].fillna(train_df.describe()["Age"]["mean"])
test_df["Age"] = test_df["Age"].fillna(test_df.describe()["Age"]["mean"])
train_df['Age']=train_df['Age'].astype(np.float64)
train_df["Fare"] = train_df["Fare"].fillna(train_df.describe()["Fare"]["mean"])
test_df["Fare"] = test_df["Fare"].fillna(test_df.describe()["Fare"]["mean"])
train_df['Fare']=train_df['Fare'].astype(int)
train_df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8,S


Here we are checking each row and extracting titles for some analysis

In [56]:
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

pd.crosstab(train_df['Title'], train_df['Sex'])

Sex,female,male
Title,,
Capt,0,1
Col,0,2
Countess,1,0
Don,0,1
Dr,1,6
Jonkheer,0,1
Lady,1,0
Major,0,2
Master,0,40


As per our above extraction we have created feature "Title"

In [57]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

train_df.head(6)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,7,S,1.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,71,C,3.0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,7,S,2.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,53,S,3.0
4,5,0,3,"Allen, Mr. William Henry",male,35.000000,0,0,8,S,1.0
5,6,0,3,"Moran, Mr. James",male,29.699118,0,0,8,Q,1.0


In [58]:
#Dropping passenger id and name column as it is not important for our model

train_df = train_df.drop(['Name', 'PassengerId'], axis=1)
test_df = test_df.drop(['Name'], axis=1)
combine = [train_df, test_df]
train_df.shape, test_df.shape

((891, 9), (418, 9))

In [59]:
#Converting 'Sex' column in binary
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,0,3,0,22.0,1,0,7,S,1.0
1,1,1,1,38.0,1,0,71,C,3.0
2,1,3,1,26.0,0,0,7,S,2.0
3,1,1,1,35.0,1,0,53,S,3.0
4,0,3,0,35.0,0,0,8,S,1.0


In [60]:
#Normalising age column and giving value within certain range
for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age']
train_df.head(6)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,0,3,0,1.0,1,0,7,S,1.0
1,1,1,1,2.0,1,0,71,C,3.0
2,1,3,1,1.0,0,0,7,S,2.0
3,1,1,1,2.0,1,0,53,S,3.0
4,0,3,0,2.0,0,0,8,S,1.0
5,0,3,0,1.0,0,0,8,Q,1.0


###  Doing some more analysis

In [61]:
for dataset in combine:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

train_df[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,FamilySize,Survived
3,4,0.724138
2,3,0.578431
1,2,0.552795
6,7,0.333333
0,1,0.303538
4,5,0.200000
5,6,0.136364
7,8,0.000000
8,11,0.000000


In [62]:
for dataset in combine:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

train_df[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean()

,IsAlone,Survived
0,0,0.505650
1,1,0.303538


In [63]:
# Dropping more column and preparing dataset for final modelling

train_df = train_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
test_df = test_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
combine = [train_df, test_df]

train_df.head(6)

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone
0,0,3,0,1.0,7,S,1.0,0
1,1,1,1,2.0,71,C,3.0,0
2,1,3,1,1.0,7,S,2.0,1
3,1,1,1,2.0,53,S,3.0,0
4,0,3,0,2.0,8,S,1.0,1
5,0,3,0,1.0,8,Q,1.0,1


In [64]:
for dataset in combine:
    dataset['Age*Class'] = dataset.Age * dataset.Pclass

train_df.loc[:, ['Age*Class', 'Age', 'Pclass']].head(10)

,Age*Class,Age,Pclass
0,3.0,1.0,3
1,2.0,2.0,1
2,3.0,1.0,3
3,2.0,2.0,1
4,6.0,2.0,3
5,3.0,1.0,3
6,3.0,3.0,1
7,0.0,0.0,3
8,3.0,1.0,3
9,0.0,0.0,2


In [65]:
freq_port = train_df.Embarked.dropna().mode()[0]
freq_port

'S'

In [66]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)
    
train_df[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Embarked,Survived
0,C,0.553571
1,Q,0.389610
2,S,0.339009


In [67]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

train_df.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone,Age*Class
0,0,3,0,1.0,7,0,1.0,0,3.0
1,1,1,1,2.0,71,1,3.0,0,2.0
2,1,3,1,1.0,7,0,2.0,1,3.0
3,1,1,1,2.0,53,0,3.0,0,2.0
4,0,3,0,2.0,8,0,1.0,1,6.0


In [85]:

#Filling fare column with null values with median price of dataset 
test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace=True)
test_df.head()


,PassengerId,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone,Age*Class
0,892,3,0,2.0,0,2,1.0,1,6.0
1,893,3,1,2.0,0,0,3.0,0,6.0
2,894,2,0,3.0,1,2,1.0,1,6.0
3,895,3,0,1.0,1,0,1.0,1,3.0
4,896,3,1,1.0,1,0,3.0,0,3.0


In [86]:
#Normalising fare column
for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

#train_df = train_df.drop(['FareBand'], axis=1)
combine = [train_df, test_df]
  
#checking for null values in dataset    
train_df.isnull().values.any()    


False

## Dividing dataset into Train and Testing

In [70]:
X_train = train_df.drop("Survived", axis=1)
Y_train = train_df["Survived"]
X_test  = test_df.drop("PassengerId", axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

((891, 8), (891,), (418, 8))

In [71]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(X_train,Y_train,test_size=.20,random_state=5)

### a)	Square impurity: 

This function will calculate impurity of the labels. 
It will take input a vector of n labels and output the corresponding squared loss impurity (below is the function)

In [88]:
def sqimpurity(yTr):
    """Computes the weighted variance of the labels
    
    Input:
        yTr:     n-dimensional vector of labels
    
    Output:
        impurity: weighted variance / squared loss impurity of this data set
    """
    
    ymean = np.mean(yTr)
    
    for y in yTr:
        impurity += (y-ymean)**2
    return impurity

### b)	Split function: 
This function will take dataset with labels as input and computes the best feature and cut value of an optimal split based on the squared error impurity we defined above. The cut value should be the average of the values in the dimension where two datapoints are split.

In order to make our life easy, we first sort the dataset features and loop through and see where two features is different and take average of those two points and calculate loss and this will do for ever feature in the dataset. For example lets say we have 5 features and and their values are 0s, 1s and 2s like [0,1,0,2,1],
after sort this will become, [0,0,1,1,2] and we will loop through this and see the change in feature value.


#### Here we are also taking help of helper function all_same to check if all the values in colum same and if it is, we continue in loop.

In [91]:
def sqsplit(xTr, yTr):
    """Finds the best feature, cut value, and loss value.
    
    Input:
        xTr:     n x d matrix of data points
        yTr:     n-dimensional vector of labels
    
    Output:
        feature:  index of the best cut's feature
        cut:      cut-value of the best cut
        bestloss: loss of the best cut
    """
    N,D = xTr.shape
    
    
    bestloss = np.inf
    feature = np.inf
    cut = np.inf

  
    idx   = np.argsort(xTr.T)
    for i, c in enumerate(xTr.T):
        list1 = np.array(c)[idx[i]]
        list2 = np.array(yTr)[idx[i]]
        if(all_same(list1)):
            continue
    
   
        for r in range(len(list1)-1):
            lftsquare=0.0
            rghtsquare =0.0
        
           
            if(list1[r] < list1[r+1]):
            
                if len(list2[:r+1])!=0:
                    lftsquare = sqimpurity(list2[:r+1])
                if len(list2[r+2:])!=0:
                    rghtsquare = sqimpurity(list2[r+1:])
               
                value = (list1[r]+list1[r+1])/2.0
              
                myloss = lftsquare + rghtsquare
                if myloss <=bestloss:
                    bestloss=myloss
                    cut = value
                    feature = i
    return feature, cut, bestloss
    



In [92]:
# this is TreeNode class
class TreeNode(object):
    """Tree class.
    (You don't need to add any methods or fields here but feel
    free to if you like. The tests will only reference the fields
    defined in the constructor below, so be sure to set these
    correctly.)
    """
    
    def __init__(self, left, right, feature, cut, prediction):
        self.left = left 
        self.right = right 
        self.feature = feature 
        self.cut = cut
        self.prediction = prediction 

### Helper function
This is our helper function to check if all the values in our list or column same or not if same it is pure make it leaf

In [93]:
def all_same(items):
    
    #print('this is items')
    #print(items)
    #print(np.all(items == items[0]))
    if len(items) > 0:
        return np.all(items == items[0])
    elif len(items) is 0:
        return False
    #print(items)
    #print(type(items))
    #print(all(x == items[0] for x in items))
    #return all(x == items[0] for x in items)

#### c) CART function: 
This function will build tree and before that it will check the base cases and stop the recursive call if any of those met or else it will keep splitting dataset.

In [94]:
def cart(xTr,yTr):
    """Builds a CART tree.
    
    The maximum tree depth is defined by "maxdepth" (maxdepth=2 means one split).
    Each example can be weighted with "weights".

    Args:
        xTr:      n x d matrix of data
        yTr:      n-dimensional vector

    Returns:
        tree: root of decision tree
    """
    n,d = xTr.shape
    maxdepth =5
    
    
    if len(yTr) == 0:
        return None
    elif all_same(yTr):
        #print("i am in same ytr")
        return TreeNode(None, None, None, None, yTr[0])
    elif len(xTr)<=2:
        #print("len is less 2")
        return TreeNode(None,None,None,None,np.mean(yTr))
    elif all_same(xTr):
        #print("i am in same all x")
        return TreeNode(None,None,None,None,majority_vote(yTr))
    else:
      
        fid, cut, loss = sqsplit(xTr,yTr)
      
        y_left = yTr[xTr[:, fid] <= cut]
        y_right = yTr[xTr[:, fid] > cut]
        
       
        x_left = xTr[xTr[:,fid]<=cut]
      
        x_right = xTr[xTr[:,fid]>cut]
       
        
        leftnode = cart(x_left,y_left)
        rightnode = cart(x_right,y_right)
        root= TreeNode(leftnode, rightnode,fid,cut,majority_vote(yTr))
  
        return root
        

In [95]:
# This funciton basically checks majority of labels in the list
def majority_vote(y):
        # this is for calculating values for the leaf nodes
    return np.bincount(y).argmax() 

### Predict function
The final function is predict, this will just take the test point and traverse the tree and get to the leaf and return the prediction. 

In [96]:
def evaltree(root,xTe):
    """Evaluates xTe using decision tree root.
    
    Input:
        root: TreeNode decision tree
        xTe:  n x d matrix of data points
    
    Output:
        pred: n-dimensional vector of predictions
    """
  
    y_pred = []
    for inputs in xTe:
        prediction =  predict(root, inputs)
        y_pred.append(prediction)
        
    
    return y_pred
    

In [80]:
def predict(root, X):
        
        """Predict class for a single sample."""
        #node = self.tree_
        while root.left:
            if X[root.feature] < root.cut:
                root = root.left
            else:
                root = root.right
        return root.prediction

### Evaluation our model on  titanic dataset

In [81]:
xtrain = xtrain.values
ytrain = ytrain.values

xtest = xtest.values
titan = cart(xtrain,ytrain)
ypred = evaltree(titan, xtest)

accuracy = np.mean(ytest == ypred)
print(accuracy*100)

84.91620111731844


## Evaluating same dataset on Sklearn's model 

In [82]:
from sklearn.tree import DecisionTreeClassifier

In [84]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(xtrain, ytrain)
Y_pred = decision_tree.predict(xtest)
acc_decision_tree = round(decision_tree.score(xtest, ytest) * 100, 2)
acc_decision_tree

84.92